In [1]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "segmentAnything"
FINAL_DATA_NAME = 'segmentAnything-full-wavelet' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "segmentAnything-croppedDeblurred"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','segmentAnything')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [3]:
file_list = [os.path.join(data_dir, f"{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"{RAW_DATA_SUFFIX}"))
file_names[:5]

['sa_7209.jpg', 'sa_2371.jpg', 'sa_4700.jpg', 'sa_3709.jpg', 'sa_5378.jpg']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
# '''To split large dataset into many batches, only needs to be run once'''
# k = 5000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'segmentAnything') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

In [6]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


segmentAnything-croppedDeblurred



In [7]:
import numpy as np
np.random.seed(42)  
def jpg_opener(path):
    
    # Apply jitter
    image = np.array(Image.open(path).convert('RGB'))
    arr = image.astype(np.float64)
    jitter = np.random.uniform(-0.5, 0.5, arr.shape)
    arr += jitter
    arr = arr - np.mean(arr)
    arr = arr / np.std(arr)

    
    return arr


# Wavelet

In [8]:
FINAL_DATA_NAME = 'segmentAnything-full-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [11]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')


    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

del channel_wv
del group_data_map
del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-733.8397 , -523.5202 , -497.3036 , ...,  477.03714,  480.7086 ,
        490.19824], dtype=float32), np.int64(3): array([-381.5477 , -326.1934 , -321.54593, ...,  320.8372 ,  342.97174,
        349.04706], dtype=float32), np.int64(4): array([-228.4368 , -201.56972, -201.24597, ...,  214.87521,  228.35588,
        272.5188 ], dtype=float32), np.int64(5): array([-283.77243 , -124.51998 , -116.712364, ...,  122.18985 ,
        130.19968 ,  204.02596 ], dtype=float32), np.int64(6): array([-126.91914 ,  -63.65492 ,  -58.643837, ...,   63.722054,
         71.40992 ,   88.32904 ], dtype=float32), np.int64(7): array([-56.944874, -32.69587 , -30.127714, ...,  30.413774,  33.974655,
        61.469116], dtype=float32), np.int64(8): array([-51.78339  , -16.095322 , -14.7365675, ...,  15.01632  ,
        16.552618 ,  41.069557 ], dtype=float32), np.int64(9): array([-36.255848 ,  -8.269207 ,  -7.485098 , ...,   7.4724035,
         8.217231 ,  19.918056 ], dtype=float3

In [12]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

del channel_wv
del group_data_map
del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-464.96548, -455.56238, -442.8803 , ...,  402.6478 ,  405.05383,
        425.76584], dtype=float32), np.int64(3): array([-307.8523 , -282.25876, -278.19205, ...,  267.09433,  280.1341 ,
        337.51926], dtype=float32), np.int64(4): array([-217.27238, -215.33658, -213.24695, ...,  194.79546,  214.85889,
        255.17145], dtype=float32), np.int64(5): array([-137.30583 , -111.144714, -107.31204 , ...,  113.94874 ,
        124.08283 ,  185.43411 ], dtype=float32), np.int64(6): array([-92.68034 , -57.744896, -55.16004 , ...,  55.34837 ,  61.13868 ,
        79.6083  ], dtype=float32), np.int64(7): array([-45.79167 , -31.009092, -28.286446, ...,  28.971672,  31.469461,
        50.34251 ], dtype=float32), np.int64(8): array([-28.716234, -15.585561, -14.379328, ...,  14.614774,  15.840999,
        34.298542], dtype=float32), np.int64(9): array([-17.980148,  -8.16033 ,  -7.417339, ...,   7.431506,   8.110897,
        18.179914], dtype=float32), np.int64(10): 

In [13]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')



    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

del channel_wv
del group_data_map
del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-610.7345 , -515.8348 , -484.62192, ...,  503.93292,  561.12604,
        565.79175], dtype=float32), np.int64(3): array([-329.62918, -319.91806, -312.8861 , ...,  293.40527,  298.4928 ,
        349.25482], dtype=float32), np.int64(4): array([-221.2627 , -214.76782, -211.10687, ...,  212.02336,  236.4869 ,
        246.3517 ], dtype=float32), np.int64(5): array([-128.5363 , -116.51817, -114.92238, ...,  119.09147,  124.08746,
        176.22348], dtype=float32), np.int64(6): array([-73.61792 , -58.264446, -55.811554, ...,  57.67349 ,  64.341705,
        99.774925], dtype=float32), np.int64(7): array([-50.291943, -31.983091, -28.517921, ...,  28.646849,  31.05483 ,
        45.23739 ], dtype=float32), np.int64(8): array([-27.120691 , -15.3747635, -14.261824 , ...,  14.495656 ,
        15.692236 ,  28.480913 ], dtype=float32), np.int64(9): array([-17.295876 ,  -8.0578375,  -7.3574514, ...,   7.310516 ,
         7.983105 ,  16.79103  ], dtype=float32), np.int64

In [14]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')


    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

del channel_wv
del group_data_map
del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-426.3494 , -400.0464 , -398.47656, ...,  412.7874 ,  415.9912 ,
        424.8862 ], dtype=float32), np.int64(3): array([-309.98468, -280.3155 , -264.45114, ...,  266.73306,  268.48187,
        333.83807], dtype=float32), np.int64(4): array([-199.80577, -197.40376, -193.0675 , ...,  195.10178,  207.13661,
        258.20688], dtype=float32), np.int64(5): array([-140.68346, -111.25855, -107.59347, ...,  115.22226,  124.06321,
        189.25455], dtype=float32), np.int64(6): array([-93.496635, -57.410706, -54.609184, ...,  55.93286 ,  60.035305,
        77.639854], dtype=float32), np.int64(7): array([-46.621693, -30.627481, -27.968359, ...,  28.414833,  30.714956,
        50.19887 ], dtype=float32), np.int64(8): array([-29.096115, -15.41388 , -14.2338  , ...,  14.447264,  15.536437,
        33.073692], dtype=float32), np.int64(9): array([-17.80166  ,  -8.124304 ,  -7.387607 , ...,   7.3680005,
         8.067773 ,  18.252169 ], dtype=float32), np.int64(10): 